In [24]:
# Import modules to be used

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras import applications
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from keras.models import load_model
from os import listdir
from os.path import join
import pandas as pd

In [2]:
img_width, img_height = 224, 224
train_data_dir = "data/train"
validation_data_dir = "data/validation"
nb_train_samples = 777 + 761
nb_validation_samples = 194 + 190
batch_size = 8
epochs = 50

In [3]:
# Load up the mobilenet architecture as a base

model = applications.MobileNet(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))

In [4]:
#Adding custom Layers 

x = model.output
x = Flatten()(x)
x = Dense(16, activation="relu")(x)
x = Dropout(0.5)(x)
x = Dense(16, activation="relu")(x)
predictions = Dense(1, activation="sigmoid")(x)

model_final = Model(model.input, predictions)

In [5]:
# Load the pretrained model
model_final.load_weights("mobilenet_1.h5")


In [6]:
datagen = ImageDataGenerator(rescale=1./255)

test_generator = datagen.flow_from_directory(
        'data',
        target_size=(img_width, img_height),
        batch_size=1,
        class_mode=None,  # this means our generator will only yield batches of data, no labels
        shuffle=False)


Found 2180 images belonging to 3 classes.


In [7]:
model_final.predict_generator(test_generator, steps=1)

array([[0.10323512]], dtype=float32)

In [8]:
prediction = model_final.predict_generator(test_generator, steps=1)

In [9]:
prediction.shape

(1, 1)

In [10]:
# Test sample with no rust in the image
# Output should be low showing low likelyhood of rust in the image
prediction

array([[0.10323512]], dtype=float32)

In [11]:
prediction = model_final.predict_generator(test_generator, steps=1)

In [12]:
# Test sample with rust in the image
# Output should be high showing high likelyhood of rust in the image
prediction

array([[0.10323512]], dtype=float32)

In [13]:
prediction = model_final.predict_generator(test_generator, steps=1)

In [14]:
# Test sample with no rust in the image
# Output should be low showing low likelyhood of rust in the image
prediction

array([[0.10323512]], dtype=float32)

In [15]:
def image_predict(image, img_width=224, img_height=224):
    img = load_img(str(image))  # this is a PIL image
    x = img.resize((img_width, img_height))
    x = img_to_array(x)  # this is a Numpy array with shape (224, 224, 3)
    x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 224, 224), 3
    x = x/255.
    return model_final.predict(x)

# img = load_img('data/train/rust/5_0052 (31) (Small).jpg')  # this is a PIL image
# x = img.resize((img_width, img_height))
# x = img_to_array(x)  # this is a Numpy array with shape (224, 224, 3)
# x = x.reshape((1,) + x.shape)  # this is a Numpy array with shape (1, 224, 224), 3
# x = x/255.

In [18]:
test_data_dir = 'data/test'

In [19]:
test_images = listdir(test_data_dir)

In [20]:
test_images

['1_0043 - 3 (Small).JPG',
 'DSCN6340 (Large).JPG',
 'DSCN6341 (Large).JPG',
 'DSCN6342 (Large).JPG',
 'DSCN6343 (Large).JPG',
 'DSCN6344 (Large).JPG',
 'DSCN6345 (Large).JPG',
 'DSCN6346 (Large).JPG',
 'DSCN6347 (Large).JPG',
 'DSCN6348 (Large).JPG',
 'DSCN6349 (Large).JPG',
 'DSCN6350 (Large).JPG',
 'DSCN6351 (Large).JPG',
 'DSCN6352 (Large).JPG',
 'DSCN6353 (Large).JPG',
 'DSCN6354 (Large).JPG',
 'DSCN6355 (Large).JPG',
 'DSCN6356 (Large).JPG',
 'DSCN6357 (Large).JPG',
 'DSCN6358 (Large).JPG',
 'DSCN6359 (Large).JPG',
 'DSCN6360 (Large).JPG',
 'DSCN6361 (Large).JPG',
 'DSCN6362 (Large).JPG',
 'DSCN6363 (Large).JPG',
 'DSCN6364 (Large).JPG',
 'DSCN6365 (Large).JPG',
 'DSCN6366 (Large).JPG',
 'DSCN6367 (Large).JPG',
 'DSCN6368 (Large).JPG',
 'DSCN6369 (Large).JPG',
 'DSCN6370 (Large).JPG',
 'DSCN6371 (Large).JPG',
 'DSCN6372 (Large).JPG',
 'DSCN6373 (Large).JPG',
 'DSCN6374 (Large).JPG',
 'DSCN6375 (Large).JPG',
 'DSCN6376 (Large).JPG',
 'DSCN6377 (Large).JPG',
 'DSCN6378 (Large).JPG'

In [26]:
full_predictions = {}

In [27]:
for f in test_images:
    pred = image_predict(join(test_data_dir, f))
    full_predictions[f] = pred.sum()

In [34]:
df = pd.DataFrame.from_dict(full_predictions, orient='index')

TypeError: from_dict() got an unexpected keyword argument 'columns'

In [37]:
df.columns = ['Prediction']

In [38]:
df.head()

,Prediction
DSCN6470 (Large).JPG,0.555495
DSCN6486 (Large).JPG,0.169566
DSCN6409 (Large).JPG,0.144349
DSCN6358 (Large).JPG,0.224340
DSCN6581 (Large).JPG,0.367570


In [39]:
rusty = df[df['Prediction'] > 0.5]
non_rusty = df[df['Prediction'] <= 0.5]

In [41]:
len(rusty['Prediction'])

17

In [44]:
rusty['Prediction']

DSCN6470 (Large).JPG        0.555495
DSCN6448 (Large).JPG        0.545749
DSCN6402 (Large).JPG        0.507073
DSCN6481 (Large).JPG        0.504599
DSCN6356 (Large).JPG        0.527333
DSCN6517 (Large).JPG        0.560456
pexels-photo-114108.jpeg    0.706852
DSCN6465 (Large).JPG        0.508932
DSCN6505 (Large).JPG        0.571130
DSCN6584 (Large).JPG        0.549203
DSCN6577 (Large).JPG        0.594816
DSCN6492 (Large).JPG        0.667496
images.jpg                  0.680249
DSCN6587 (Large).JPG        0.502152
DSCN6516 (Large).JPG        0.539113
DSCN6369 (Large).JPG        0.527210
DSCN6455 (Large).JPG        0.533608
Name: Prediction, dtype: float64

In [43]:
len(non_rusty['Prediction'])

241

In [9]:
rusty_1 = image_predict('data/test/images.jpg')
rusty_2 = image_predict('data/test/pexels-photo-114108.jpeg')
non_rusty_1 = image_predict('data/test/no_rust.jpg')
non_rusty_2 = image_predict('data/test/no_rust_2.jpg')
non_rusty_3 = image_predict('data/test/1_0043 - 3 (Small).JPG')

In [11]:
print('Rusty 1 score:', rusty_1.sum())
print('Rusty 2 score:', rusty_2.sum())
print('Non_rusty 1 score:', non_rusty_1.sum())
print('Non_rusty 2 score:', non_rusty_2.sum())
print('Non_rusty 3 score:', non_rusty_3.sum())

Rusty 1 score: 0.68024904
Rusty 2 score: 0.7068523
Non_rusty 1 score: 0.16616271
Non_rusty 2 score: 0.03634391
Non_rusty 3 score: 0.10323502


### Now test the newly trained Mobilenet_V2

In [13]:
base_model = applications.mobilenetv2.MobileNetV2(include_top=False, input_shape = (img_width, img_height, 3), pooling='avg')

output = Dense(1, activation='sigmoid')(base_model.output)
model_final = Model(base_model.input, output)

In [14]:
# Load the pretrained model
model_final.load_weights("Mobile_V2.h5")

In [15]:
rusty_1 = image_predict('data/test/images.jpg')
rusty_2 = image_predict('data/test/pexels-photo-114108.jpeg')
non_rusty_1 = image_predict('data/test/no_rust.jpg')
non_rusty_2 = image_predict('data/test/no_rust_2.jpg')
non_rusty_3 = image_predict('data/test/1_0043 - 3 (Small).JPG')

In [16]:
print('Rusty 1 score:', rusty_1.sum())
print('Rusty 2 score:', rusty_2.sum())
print('Non_rusty 1 score:', non_rusty_1.sum())
print('Non_rusty 2 score:', non_rusty_2.sum())
print('Non_rusty 3 score:', non_rusty_3.sum())

Rusty 1 score: 0.7832331
Rusty 2 score: 0.4784979
Non_rusty 1 score: 0.37875006
Non_rusty 2 score: 0.1442362
Non_rusty 3 score: 0.059132084


### Test out newly trained Xception net

Firstly load up the Xception net in keras without the top layer, and add a final Dense output layer

In [47]:
base_model = applications.Xception(include_top=False, input_shape = (img_width, img_height, 3), pooling='avg')

output = Dense(1, activation='sigmoid')(base_model.output)
model_final = Model(base_model.input, output)

Once the model is loaded, load in the pretrained weights from the neural net training

In [48]:
# Load the pretrained model
model_final.load_weights("Xception_1.h5")

In [49]:
full_predictions = {}
for f in test_images:
    pred = image_predict(join(test_data_dir, f))
    full_predictions[f] = pred.sum()

In [50]:
df = pd.DataFrame.from_dict(full_predictions, orient='index')

In [51]:
df.head()

,0
DSCN6470 (Large).JPG,0.990169
DSCN6486 (Large).JPG,0.165428
DSCN6409 (Large).JPG,0.256433
DSCN6358 (Large).JPG,0.060973
DSCN6581 (Large).JPG,0.000090


In [53]:
df.columns = ['Prediction']

In [54]:
rusty = df[df['Prediction'] > 0.5]
non_rusty = df[df['Prediction'] <= 0.5]

In [55]:
len(rusty['Prediction'])

12

In [56]:
rusty['Prediction']

DSCN6470 (Large).JPG        0.990169
DSCN6402 (Large).JPG        0.993661
DSCN6374 (Large).JPG        0.556375
DSCN6380 (Large).JPG        0.665090
DSCN6379 (Large).JPG        0.931861
DSCN6382 (Large).JPG        0.850072
pexels-photo-114108.jpeg    0.996258
DSCN6420 (Large).JPG        0.980705
DSCN6479 (Large).JPG        0.668909
DSCN6492 (Large).JPG        0.600438
images.jpg                  0.999717
DSCN6557 (Large).JPG        0.646895
Name: Prediction, dtype: float64

Test the neural net by predicting on the test sample set of images.

The test set has two images showing rust and 3 images showing no rust. The predictions for the images showing rust should be close to 1 and the images showing no rust should be close to 0.

This set of images has been downloaded from google images and is completely different from the training and validation data sets.

In [6]:
rusty_1 = image_predict('data/test/images.jpg')
rusty_2 = image_predict('data/test/pexels-photo-114108.jpeg')
non_rusty_1 = image_predict('data/test/no_rust.jpg')
non_rusty_2 = image_predict('data/test/no_rust_2.jpg')
non_rusty_3 = image_predict('data/test/1_0043 - 3 (Small).JPG')

In [7]:
print('Rusty 1 score:', rusty_1.sum())
print('Rusty 2 score:', rusty_2.sum())
print('Non_rusty 1 score:', non_rusty_1.sum())
print('Non_rusty 2 score:', non_rusty_2.sum())
print('Non_rusty 3 score:', non_rusty_3.sum())

Rusty 1 score: 0.9997166
Rusty 2 score: 0.9962578
Non_rusty 1 score: 0.14408405
Non_rusty 2 score: 0.003509822
Non_rusty 3 score: 5.661761e-06


As can be seen from the scores predicted above the neural net has correctly classified the images and not only that has predicted scores very close to 1 for the rusty images, and very close to 0 for the images without rust.

### Now test the updated trained Mobilenet_V2

In [57]:
base_model = applications.mobilenetv2.MobileNetV2(include_top=False, input_shape = (img_width, img_height, 3), pooling='avg')

output = Dense(1, activation='sigmoid')(base_model.output)
model_final = Model(base_model.input, output)

In [58]:
# Load the pretrained model
model_final.load_weights("Mobile_test.h5")

In [59]:
full_predictions = {}
for f in test_images:
    pred = image_predict(join(test_data_dir, f))
    full_predictions[f] = pred.sum()

In [60]:
df = pd.DataFrame.from_dict(full_predictions, orient='index')

In [61]:
df.columns = ['Prediction']

In [62]:
rusty = df[df['Prediction'] > 0.5]
non_rusty = df[df['Prediction'] <= 0.5]

In [63]:
len(rusty['Prediction'])

14

In [64]:
rusty['Prediction']

DSCN6470 (Large).JPG        0.736087
DSCN6514 (Large).JPG        0.601293
DSCN6406 (Large).JPG        0.781048
pexels-photo-114108.jpeg    0.971214
DSCN6578 (Large).JPG        0.563262
DSCN6420 (Large).JPG        0.625713
DSCN6541 (Large).JPG        0.570009
DSCN6492 (Large).JPG        0.846626
DSCN6508 (Large).JPG        0.729443
images.jpg                  0.807393
DSCN6535 (Large).JPG        0.503547
DSCN6589 (Large).JPG        0.798120
DSCN6450 (Large).JPG        0.679698
DSCN6400 (Large).JPG        0.563906
Name: Prediction, dtype: float64

In [6]:
rusty_1 = image_predict('data/test/images.jpg')
rusty_2 = image_predict('data/test/pexels-photo-114108.jpeg')
non_rusty_1 = image_predict('data/test/no_rust.jpg')
non_rusty_2 = image_predict('data/test/no_rust_2.jpg')
non_rusty_3 = image_predict('data/test/1_0043 - 3 (Small).JPG')

In [7]:
print('Rusty 1 score:', rusty_1.sum())
print('Rusty 2 score:', rusty_2.sum())
print('Non_rusty 1 score:', non_rusty_1.sum())
print('Non_rusty 2 score:', non_rusty_2.sum())
print('Non_rusty 3 score:', non_rusty_3.sum())

Rusty 1 score: 0.80739266
Rusty 2 score: 0.9712144
Non_rusty 1 score: 0.36098155
Non_rusty 2 score: 0.21638988
Non_rusty 3 score: 0.010145643


All test images have been classified correctly which is an improvement from the first iteration of the mobilenet.

In [6]:
model_final.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (Activation)         (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
__________________________________________________________________________________________________
expanded_c